# Most Used Functions in StyleGAN

StyleGAN is a type of Generative Adversarial Network (GAN) known for its ability to generate high-quality images. It introduces the concepts of the mapping network and the synthesis network, along with style mixing and adaptive instance normalization (AdaIN). In this notebook, we will cover some of the most commonly used functions and techniques for implementing a simplified version of StyleGAN using TensorFlow and Keras.

## 1. Building the Mapping Network

The mapping network in StyleGAN is responsible for mapping the latent code \( z \) to an intermediate latent space \( w \). This network consists of several fully connected layers.

In [ ]:
import numpy as np
from tensorflow.keras.layers import Dense, LeakyReLU, Input
from tensorflow.keras.models import Model
import tensorflow as tf                

# Function to build the mapping network
def build_mapping_network(latent_dim):
    model = tf.keras.Sequential()
    model.add(Dense(512, input_dim=latent_dim, activation='relu'))
    for _ in range(7):
        model.add(Dense(512, activation='relu'))
    return model

# Instantiate and summarize the mapping network
latent_dim = 100
mapping_network = build_mapping_network(latent_dim)
mapping_network.summary()

## 2. Building the Synthesis Network

The synthesis network in StyleGAN generates images from the intermediate latent space \( w \). It progressively grows in resolution by adding layers.

In [ ]:
from tensorflow.keras.layers import Conv2D, Conv2DTranspose, Reshape, Activation, BatchNormalization, Add

# Function to build the synthesis network
def build_synthesis_network():
    model = tf.keras.Sequential()
    model.add(Dense(4*4*512, activation='relu', input_dim=512))
    model.add(Reshape((4, 4, 512)))
    model.add(Conv2DTranspose(256, (3, 3), strides=(2, 2), padding='same', activation='relu'))
    model.add(Conv2D(256, (3, 3), padding='same', activation='relu'))
    model.add(Conv2DTranspose(128, (3, 3), strides=(2, 2), padding='same', activation='relu'))
    model.add(Conv2D(128, (3, 3), padding='same', activation='relu'))
    model.add(Conv2DTranspose(64, (3, 3), strides=(2, 2), padding='same', activation='relu'))
    model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
    model.add(Conv2DTranspose(32, (3, 3), strides=(2, 2), padding='same', activation='relu'))
    model.add(Conv2D(3, (3, 3), padding='same', activation='tanh'))
    return model

# Instantiate and summarize the synthesis network
synthesis_network = build_synthesis_network()
synthesis_network.summary()

## 3. Building the Discriminator

The discriminator in StyleGAN is similar to that in other GANs, distinguishing between real and fake images. It progressively reduces the resolution by adding convolutional layers.

In [ ]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Flatten

# Function to build the discriminator
def build_discriminator(input_shape):
    model = tf.keras.Sequential()
    model.add(Conv2D(64, (3, 3), strides=(2, 2), padding='same', input_shape=input_shape))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Conv2D(128, (3, 3), strides=(2, 2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Conv2D(256, (3, 3), strides=(2, 2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Conv2D(512, (3, 3), strides=(2, 2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Flatten())
    model.add(Dense(1, activation='sigmoid'))
    return model

# Instantiate and summarize the discriminator
discriminator = build_discriminator((32, 32, 3))
discriminator.compile(optimizer=Adam(0.0002, 0.5), loss='binary_crossentropy', metrics=['accuracy'])
discriminator.summary()

## 4. Building and Compiling the StyleGAN

The StyleGAN combines the mapping network, synthesis network, and discriminator. The generator network consists of the mapping and synthesis networks. The GAN is trained using the combined model.

In [ ]:
# Build and compile the StyleGAN
def build_stylegan(generator, discriminator):
    discriminator.trainable = False
    model = tf.keras.Sequential([generator, discriminator])
    model.compile(optimizer=Adam(0.0002, 0.5), loss='binary_crossentropy')
    return model

# Create inputs
z = Input(shape=(latent_dim,))
w = mapping_network(z)
img = synthesis_network(w)

# Create generator model
generator = Model(z, img)
generator.summary()

# Create and compile GAN model
stylegan = build_stylegan(generator, discriminator)
stylegan.summary()